### Linear and Multilinear Functions

In [1]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
alg = Algebra(1,3)
locals().update(alg.blades)

In [2]:
#1.1 linear transformation
A = create_random_multivector(alg)
B = create_random_multivector(alg)
x = create_random_multivector(alg).grade(1)
a = create_random_multivector(alg).grade(1)
f = lambda x: (A*x*B).grade(1)
differential(f, x, a), f(a)

(-4.0 𝐞₁ + -2.34 𝐞₂ + -1.9 𝐞₃ + 3.11 𝐞₄,
 -4.0 𝐞₁ + -2.34 𝐞₂ + -1.9 𝐞₃ + 3.11 𝐞₄)

In [3]:
#1.2
C = create_random_multivector(alg)
D = create_random_multivector(alg)
g = lambda x: (C*x*D).grade(1)
g(f(a)), differential(g, f(a), f(a))

(-1.32 𝐞₁ + 1.83 𝐞₂ + -4.45 𝐞₃ + -1.5 𝐞₄,
 -1.32 𝐞₁ + 1.83 𝐞₂ + -4.45 𝐞₃ + -1.5 𝐞₄)

In [4]:
def skew_symmetrizer(F, Ar, alg, h=1e-6, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(alg.frame)
    drF = 0
    r = Ar.grades[0]
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # if F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # So actually we allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, np.zeros(r), reci_vectors, h=h)
    return (1/factorial(r)) * drF

In [5]:
#1.4
def outermorphism(f, A: MultiVector, alg, h=1e-6, frame=None, r_frame=None):
    F = lambda vectors: wedge([f(v) for v in vectors])
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        outer += skew_symmetrizer(F, A.grade(r), alg, h, frame, r_frame)
    return outer

outermorphism(f, x, alg), f(x)

(-3.69 𝐞₁ + -2.67 𝐞₂ + -0.349 𝐞₃ + 1.83 𝐞₄,
 -3.69 𝐞₁ + -2.67 𝐞₂ + -0.349 𝐞₃ + 1.83 𝐞₄)

In [6]:
#1.6
h = 1e-2
A2 = e12
A2frame = [e1, e2]
A2r_frame = reciprocal(A2frame)

outermorphism(f, P(A, A2), alg, h, frame=A2frame, r_frame=A2r_frame), outermorphism(f, A, alg, h, frame=A2frame, r_frame=A2r_frame)

(0.496 + -0.812 𝐞₁ + -0.391 𝐞₂ + -0.53 𝐞₃ + 0.886 𝐞₄ + -0.445 𝐞₁₂ + 3.14 𝐞₁₃ + -0.304 𝐞₁₄ + 1.8 𝐞₂₃ + -0.632 𝐞₂₄ + 3.23 𝐞₃₄,
 0.496 + -0.812 𝐞₁ + -0.391 𝐞₂ + -0.53 𝐞₃ + 0.886 𝐞₄ + -0.445 𝐞₁₂ + 3.14 𝐞₁₃ + -0.304 𝐞₁₄ + 1.8 𝐞₂₃ + -0.632 𝐞₂₄ + 3.23 𝐞₃₄)

In [7]:
#1.7
vectors = create_r_vectors(3, alg)
outermorphism(f, wedge(vectors), alg), wedge([f(v) for v in vectors])

(-0.727 𝐞₁₂₃ + -0.387 𝐞₁₂₄ + 3.16 𝐞₁₃₄ + 2.62 𝐞₂₃₄,
 -0.727 𝐞₁₂₃ + -0.387 𝐞₁₂₄ + 3.16 𝐞₁₃₄ + 2.62 𝐞₂₃₄)

In [8]:
#By 1.7, splitting A into simple blades, we get an easier implement of 1.4
def outermorphism_(f, A: MultiVector, alg):
    outer = A.grade(0)
    for vectors in multi_frame_vectors(alg.frame):
        Vr = wedge(reciprocal(vectors)[::-1])
        outer += alg.sp(Vr,A) * wedge([f(v) for v in vectors]) # frame split of A
    return outer

outermorphism_(f, D, alg), outermorphism(f, D, alg)

(0.638 + -3.49 𝐞₁ + -2.43 𝐞₂ + -1.23 𝐞₃ + 1.6 𝐞₄ + 0.313 𝐞₁₂ + -3.1 𝐞₁₃ + 0.0619 𝐞₁₄ + -2.7 𝐞₂₃ + 0.0477 𝐞₂₄ + -0.043 𝐞₃₄ + 0.718 𝐞₁₂₃ + -0.0928 𝐞₁₂₄ + -0.903 𝐞₁₃₄ + -0.94 𝐞₂₃₄ + -0.265 𝐞₁₂₃₄,
 0.638 + -3.49 𝐞₁ + -2.43 𝐞₂ + -1.23 𝐞₃ + 1.6 𝐞₄ + 0.313 𝐞₁₂ + -3.1 𝐞₁₃ + 0.0619 𝐞₁₄ + -2.7 𝐞₂₃ + 0.0477 𝐞₂₄ + -0.043 𝐞₃₄ + 0.718 𝐞₁₂₃ + -0.0928 𝐞₁₂₄ + -0.903 𝐞₁₃₄ + -0.94 𝐞₂₃₄ + -0.265 𝐞₁₂₃₄)

In [9]:
#1.9
outermorphism(f, A.reverse(), alg), outermorphism(f, A, alg).reverse()

(0.496 + -0.949 𝐞₁ + -0.496 𝐞₂ + -0.537 𝐞₃ + 0.929 𝐞₄ + -1.39 𝐞₁₂ + 2.95 𝐞₁₃ + -3.75 𝐞₁₄ + 2.8 𝐞₂₃ + -3.2 𝐞₂₄ + -0.878 𝐞₃₄ + -1.04 𝐞₁₂₃ + -0.0841 𝐞₁₂₄ + 3.58 𝐞₁₃₄ + 2.93 𝐞₂₃₄ + -0.805 𝐞₁₂₃₄,
 0.496 + -0.949 𝐞₁ + -0.496 𝐞₂ + -0.537 𝐞₃ + 0.929 𝐞₄ + -1.39 𝐞₁₂ + 2.95 𝐞₁₃ + -3.75 𝐞₁₄ + 2.8 𝐞₂₃ + -3.2 𝐞₂₄ + -0.878 𝐞₃₄ + -1.04 𝐞₁₂₃ + -0.0841 𝐞₁₂₄ + 3.58 𝐞₁₃₄ + 2.93 𝐞₂₃₄ + -0.805 𝐞₁₂₃₄)

In [10]:
#1.10
f = lambda x: P((A*x*B).grade(1), A2)
outermorphism(f, A, alg), P(outermorphism(f, A, alg), A2)

(0.496 + -0.949 𝐞₁ + -0.496 𝐞₂ + 1.39 𝐞₁₂,
 0.496 + -0.949 𝐞₁ + -0.496 𝐞₂ + 1.39 𝐞₁₂)

In [11]:
#1.11
def adjoint_outermorphism(f, A, alg, h=1e-6, frame=None, r_frame=None):
    F = lambda vectors: wedge([f(v) for v in vectors]).sp(A)
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
    return outer

In [12]:
#1.12
adjoint_outermorphism(f, P(A, A2), alg), adjoint_outermorphism(f, A, alg)

(0.496 + -0.851 𝐞₁ + -0.519 𝐞₂ + 0.373 𝐞₃ + 0.443 𝐞₄ + -0.445 𝐞₁₂ + 1.0 𝐞₁₃ + 0.601 𝐞₁₄ + 0.418 𝐞₂₃ + 0.135 𝐞₂₄ + 0.26 𝐞₃₄,
 0.496 + -0.851 𝐞₁ + -0.519 𝐞₂ + 0.373 𝐞₃ + 0.443 𝐞₄ + -0.445 𝐞₁₂ + 1.0 𝐞₁₃ + 0.601 𝐞₁₄ + 0.418 𝐞₂₃ + 0.135 𝐞₂₄ + 0.26 𝐞₃₄)

In [13]:
#1.13 also suggest we can implement the adjoint outermorphism by spliting A into vectors
adjoint_outermorphism(f, A^B, alg), wedge([adjoint_outermorphism(f, V, alg) for V in [A, B]])

(0.257 + -0.79 𝐞₁ + -0.419 𝐞₂ + 0.203 𝐞₃ + 0.326 𝐞₄ + -0.723 𝐞₁₂ + 1.63 𝐞₁₃ + 0.978 𝐞₁₄ + 0.679 𝐞₂₃ + 0.22 𝐞₂₄ + 0.422 𝐞₃₄,
 0.257 + -0.79 𝐞₁ + -0.419 𝐞₂ + 0.203 𝐞₃ + 0.326 𝐞₄ + -0.723 𝐞₁₂ + 1.63 𝐞₁₃ + 0.978 𝐞₁₄ + 0.679 𝐞₂₃ + 0.22 𝐞₂₄ + 0.422 𝐞₃₄ + -1.11e-16 𝐞₁₂₃ + -5.41e-16 𝐞₁₂₄ + 3.33e-16 𝐞₁₃₄ + 9.71e-17 𝐞₂₃₄ + -4.44e-16 𝐞₁₂₃₄)

In [14]:
#1.14a
r, s = 2, 3
f = lambda x: (A*x.grade(1)*B).grade(1)
Ar = A.grade(r)
Bs = B.grade(s)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
    adjoint_outermorphism(f, outermorphism(f, Ar, alg) | Bs, alg),
)

(4.7 𝐞₁ + 6.78 𝐞₂ + -2.95 𝐞₃ + -0.981 𝐞₄,
 -2.61 𝐞₁ + -2.38 𝐞₂ + -2.5 𝐞₃ + -1.1 𝐞₄,
 4.7 𝐞₁ + 6.78 𝐞₂ + -2.95 𝐞₃ + -0.981 𝐞₄)

In [15]:
#1.14b
r, s = 3, 2
Ar = A.grade(r)
Bs = B.grade(s)
(
    outermorphism(f, Ar | adjoint_outermorphism(f, Bs, alg), alg),
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
)

(2.86 𝐞₁ + 2.38 𝐞₂ + -0.189 𝐞₃ + 0.116 𝐞₄,
 0.85 𝐞₁ + 0.67 𝐞₂ + 0.331 𝐞₃ + -2.66 𝐞₄,
 2.86 𝐞₁ + 2.38 𝐞₂ + -0.189 𝐞₃ + 0.116 𝐞₄)

In [16]:
#1.15
for r,s in [(2,2), (3,3)]:
    Ar = A.grade(r)
    Bs = B.grade(s)
    print(
        Ar | adjoint_outermorphism(f, Bs, alg),
        outermorphism(f, Ar, alg) | Bs,
    )

-0.648 -0.648
2.39 2.39


In [17]:
# proof of 1.14a
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
frame = alg.frame
r_frame = reciprocal(alg.frame)
rhs = 0
r = Ar.grades[0]
F = lambda vectors: wedge([f(v) for v in vectors]) 
for base_vectors, reci_vectors in zip(permutations(frame, s), permutations(r_frame, s)):
    rhs += (Ar | (wedge(base_vectors[::-1]))) * (vectors_partial(F, np.zeros(s), reci_vectors, h=h) | Bs)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    rhs / factorial(s)
)

(4.7 𝐞₁ + 6.78 𝐞₂ + -2.95 𝐞₃ + -0.981 𝐞₄,
 4.7 𝐞₁ + 6.78 𝐞₂ + -2.95 𝐞₃ + -0.981 𝐞₄)

In [18]:
# singular, nonsingular functions
I = wedge(alg.frame)
fA2 = lambda x: P(f(x), A2) # a singular one
outermorphism(fA2, I, alg), outermorphism(f, I, alg), adjoint_outermorphism(f, I, alg)

(, -0.825 𝐞₁₂₃₄, -0.825 𝐞₁₂₃₄)

In [19]:
#1.16 Use inverse instead of reverse for negative signatures.
n = 4
Ir = I.inv()
F = lambda vectors: wedge([f(v) for v in vectors])
simplicial_derivative(F, alg.frame, alg, h=1e-2), Ir * outermorphism(f, I, alg), adjoint_outermorphism(f, I, alg) * Ir.reverse()

(-0.825, -0.825, -0.825)

In [45]:
#1.16 Use inverse instead of reverse for negative signatures.
n = 2
I = e12
Ip = e23
Ir = I.inv()
Ipr = Ip.inv()
h = 1e-2
# a function from I to Ip
g = lambda x: P((A*P(x.grade(1), I)*B).grade(1), Ip)
F = lambda vectors: wedge([g(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame[:2], alg, h),
    Ir * outermorphism(g, I, alg, h),
    adjoint_outermorphism(g, Ip, alg, h) * Ipr
)

(-3.33 𝐞₁₃, -3.33 𝐞₁₃, -3.33 𝐞₁₃)

In [46]:
#1.17a
outermorphism(g, I, alg, h), Ip

(3.33 𝐞₂₃, 1 𝐞₂₃)

In [47]:
#1.17b need extra minus sign for signature (3,1)
adjoint_outermorphism(g, Ip, alg, h), I

(-3.33 𝐞₁₂, 1 𝐞₁₂)

In [48]:
# The relative orientation between I and I' can be arbitrary. Here we let the parent algebra determine the orientation.
def det(f, I, Ip):
    return (outermorphism(f, I, alg)/Ip)[0]

det(g, I, Ip)

3.334188832212731

In [49]:
#1.21a null part goes null
C = create_random_multivector(alg)
C_ = outermorphism(g, C, alg)
(-1/det(g, I, Ip)) * adjoint_outermorphism(g, C_*Ip, alg) * Ir, C

(0.968 + 0.737 𝐞₁ + 0.491 𝐞₂ + 0.909 𝐞₁₂,
 0.968 + 0.737 𝐞₁ + 0.491 𝐞₂ + 0.755 𝐞₃ + 0.646 𝐞₄ + 0.909 𝐞₁₂ + 0.416 𝐞₁₃ + 0.0788 𝐞₁₄ + 0.155 𝐞₂₃ + 0.962 𝐞₂₄ + 0.873 𝐞₃₄ + 0.373 𝐞₁₂₃ + 0.126 𝐞₁₂₄ + 0.643 𝐞₁₃₄ + 0.604 𝐞₂₃₄ + 0.73 𝐞₁₂₃₄)

In [50]:
#1.21b
C = adjoint_outermorphism(g, C_, alg)
(1/det(g, I, Ip)) * outermorphism(g, C*I, alg) * Ipr, C_

(0.968 + -0.56 𝐞₂ + -1.48 𝐞₃ + 3.03 𝐞₂₃,
 0.968 + -0.56 𝐞₂ + -1.48 𝐞₃ + 3.03 𝐞₂₃)

In [51]:
#2.1
r = 3
h = 1e-2
vectors = create_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
(
    simplicial_derivative(F, vectors, alg, h),
    simplicial_derivative(lambda vectors: outermorphism(f, wedge(vectors), alg, h), vectors, alg, h),
    derivative(lambda A: outermorphism(f, A, alg, h), wedge(vectors), alg, h, r)    
)


(5.17 + 0.98 𝐞₁₂ + 5.57 𝐞₁₃ + -5.17 𝐞₁₄ + 6.47 𝐞₂₃ + -5.65 𝐞₂₄ + 1.1 𝐞₃₄,
 5.17 + 0.98 𝐞₁₂ + 5.57 𝐞₁₃ + -5.17 𝐞₁₄ + 6.47 𝐞₂₃ + -5.65 𝐞₂₄ + 1.1 𝐞₃₄,
 5.17 + 0.98 𝐞₁₂ + 5.57 𝐞₁₃ + -5.17 𝐞₁₄ + 6.47 𝐞₂₃ + -5.65 𝐞₂₄ + 1.1 𝐞₃₄)

In [52]:
#2.2
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
images = [f(v) for v in frame]
(
    sum(vr * b for b, vr in zip(images, r_frame)),
    derivative(f, x, alg)
)

(-0.621 + 0.403 𝐞₁₂ + -2.98 𝐞₁₃ + 1.62 𝐞₁₄ + -0.457 𝐞₂₃ + 0.732 𝐞₂₄ + -0.953 𝐞₃₄,
 -0.621 + 0.403 𝐞₁₂ + -2.98 𝐞₁₃ + 1.62 𝐞₁₄ + -0.457 𝐞₂₃ + 0.732 𝐞₂₄ + -0.953 𝐞₃₄)

In [53]:
#2.3 trace
def trace(f, alg):
    return sum(vr | f(v) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
(
    derivative(f, e1, alg).grade(0),
    trace(f, alg)
)

(-0.621, -0.621)

In [54]:
#2.4
r = 3
h = 1e-2
vectors = create_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
r_vec_frame = r_vector_frame_vectors(frame, r)
r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
sdf = 0
for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
    sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
(
    sdf,
    simplicial_derivative(F, vectors, alg, h)
)

(5.17 + 0.98 𝐞₁₂ + 5.57 𝐞₁₃ + -5.17 𝐞₁₄ + 6.47 𝐞₂₃ + -5.65 𝐞₂₄ + 1.1 𝐞₃₄,
 5.17 + 0.98 𝐞₁₂ + 5.57 𝐞₁₃ + -5.17 𝐞₁₄ + 6.47 𝐞₂₃ + -5.65 𝐞₂₄ + 1.1 𝐞₃₄)

In [135]:
#2.4
r = 3
h = 1e-2
vectors = create_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
frame = alg.frame
r_frame = reciprocal(frame)
r_vec_frame = r_vector_frame_vectors(frame, r)
r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
sdf = 0
for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
    sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
(
    sdf,
    simplicial_derivative(F, vectors, alg, h)
)

(-0.0815 + -0.0467 𝐞₁₂ + -0.0576 𝐞₁₃ + 0.0495 𝐞₁₄ + -0.0131 𝐞₂₃ + 0.0352 𝐞₂₄ + 0.0296 𝐞₃₄,
 -0.0815 + -0.0467 𝐞₁₂ + -0.0576 𝐞₁₃ + 0.0495 𝐞₁₄ + -0.0131 𝐞₂₃ + 0.0352 𝐞₂₄ + 0.0296 𝐞₃₄)

In [136]:
def char_multi(f, r, alg: Algebra):
    frame = alg.frame
    r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
    return sdf

char_multi(f, r, alg)

-0.0815 + -0.0467 𝐞₁₂ + -0.0576 𝐞₁₃ + 0.0495 𝐞₁₄ + -0.0131 𝐞₂₃ + 0.0352 𝐞₂₄ + 0.0296 𝐞₃₄

In [63]:
#2.6
matrix = np.array([[(ar|f(a))[0] for a in alg.frame] for ar in reciprocal(alg.frame)])
I = wedge(alg.frame)
char_multi(f, 4, alg)[0], np.linalg.det(matrix), det(f, I, I)

(-0.8251540275247926, -0.8251540275247937, -0.8251540275247865)

In [146]:
#2.7
from scipy.special import comb
r = 3
h = lambda x: f(x)+g(x)

# Correct implemetation means frame independence
# generate a random frame again
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
drF = 0
coef = [1/(factorial(r-s)*factorial(s)) for s in range(r+1)]
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    gh = 0
    for s in range(r+1):
        gh += coef[s]*wedge([f(v) for v in reci_vectors[:s]]) ^ wedge([g(v) for v in reci_vectors[s:]])
    drF += wedge(base_vectors[::-1]) * gh
char_multi(h, r, alg), drF

(-0.29 + -0.112 𝐞₁₂ + -0.16 𝐞₁₃ + 0.198 𝐞₁₄ + -0.0261 𝐞₂₃ + 0.0764 𝐞₂₄ + 0.0567 𝐞₃₄,
 -0.29 + -0.112 𝐞₁₂ + -0.16 𝐞₁₃ + 0.198 𝐞₁₄ + -0.0261 𝐞₂₃ + 0.0764 𝐞₂₄ + 0.0567 𝐞₃₄)

In [148]:
#2.8
l = 3.3
n = 4
char_multi(lambda x: f(x) - l*x, n, alg), sum((-l)**(n - s) * char_multi(f, s, alg) for s in range(n+1))

(91.6,
 91.6 + -7.08 𝐞₁₂ + -21.0 𝐞₁₃ + 14.0 𝐞₁₄ + 3.78 𝐞₂₃ + 2.28 𝐞₂₄ + 14.2 𝐞₃₄ + -4.53e-16 𝐞₁₂₃₄)

In [164]:
def r_fold(f,r,value):
    return reduce(lambda v, _: f(v), range(r), value)

r_fold(f, 3, x), f(f(f(x))), r_fold(f, 0, x), x

(0.0565 𝐞₁ + 0.0445 𝐞₂ + 0.0401 𝐞₃ + 0.0425 𝐞₄,
 0.0565 𝐞₁ + 0.0445 𝐞₂ + 0.0401 𝐞₃ + 0.0425 𝐞₄,
 0.369 𝐞₁ + 0.12 𝐞₂ + 0.968 𝐞₃ + 0.684 𝐞₄,
 0.369 𝐞₁ + 0.12 𝐞₂ + 0.968 𝐞₃ + 0.684 𝐞₄)

In [171]:
#2.9
n = 4
ch = 0
for s in range(0,n+1):
    ch += ((-1)**(n-s))*extract_scalar(char_multi(f, s, alg))*r_fold(f, n-s, x)
ch

-3.04e-18 𝐞₁ + 1.28e-17 𝐞₂ + -3.9e-18 𝐞₃ + -6.29e-18 𝐞₄

In [176]:
#2.10
matrix = np.array([[(ar|f(a))[0] for a in alg.frame] for ar in reciprocal(alg.frame)])
eigvalues, eigvecs = np.linalg.eig(matrix)
for l in eigvalues:
    print(sum(((-l)**(n-s))*extract_scalar(char_multi(f, s, alg)) for s in range(n+1)))

-1.491862189340054e-16
-1.3877787807814457e-17
-2.168404344971009e-18
4.163336342344337e-17


In [242]:
# recovering matrix to transfromation
def matrix2trans(M, alg:Algebra ):
    d = alg.d
    assert M.shape[0] == d, 'dimension not fit'
    return lambda x: sum(c*e for c,e in zip(np.dot(M, x.asfullmv()[1:d+1]), alg.frame))
    
fm = matrix2trans(matrix, alg)
fm(x), f(x)

(0.274 𝐞₁ + 0.303 𝐞₂ + 0.121 𝐞₃ + 0.33 𝐞₄,
 0.274 𝐞₁ + 0.303 𝐞₂ + 0.121 𝐞₃ + 0.33 𝐞₄)

In [185]:
#2.11
s = 2
rhs = (1/s) * sum((-1)**(r+1) * extract_scalar(char_multi(f, s-r, alg)) * trace(lambda x: r_fold(f, r, x), alg)[0] for r in range(1,s+1))
char_multi(f, s, alg)[0], rhs

(-0.06696381545784844, -0.06696381545784841)

In [179]:
#2.13
k = 3
char_multi(f, k, alg)[0], sum(np.prod(comb) for comb in combinations(eigvalues, k))

(-0.08149140824435848, -0.08149140824435847)

In [187]:
#2.14 It's easier to show the eigenvalues of f^k are {lambda^k_i} by linearity
trace(lambda x: r_fold(f, k, x), alg)[0], sum(l**k for l in eigvalues)

(0.30535808203709597, 0.305358082037095)